<img alt="QuantRocket logo" src="https://www.quantrocket.com/assets/img/notebook-header-logo.png">

<a href="https://www.quantrocket.com/disclaimer/">Disclaimer</a>

# Data Collection - Indexes

Though most of our features will come from the Sharadar price and fundamental data, we also wish to add some additional features that reflect the broad market. Specifically, we will include features relating to the S&P 500, the VIX, and the NYSE TRIN (aka Arms Index, a breadth measure). 

First, start IB Gateway:

In [1]:
from quantrocket.launchpad import start_gateways
start_gateways(wait=True)

{'ibg1': {'status': 'running'}}

## Collect listings

Next we collect the listings from IB. (For the S&P 500, we collect the SPY ETF because IB provides deeper history for SPY@ARCA than SPX@CBOE.)

In [2]:
from quantrocket.master import collect_listings
collect_listings(exchanges="ARCA", symbols="SPY", sec_types="ETF")
collect_listings(exchanges="CBOE", symbols="VIX", sec_types="IND")
collect_listings(exchanges="NYSE", symbols="TRIN-NYSE", sec_types="IND")

{'status': 'the listing details will be collected asynchronously'}

Monitor flightlog for the completion messages:

```
quantrocket.master: INFO Collecting ARCA ETF listings from IB website (SPY only)
quantrocket.master: INFO Requesting details for 1 ARCA listings found on IB website
quantrocket.master: INFO Saved 1 ARCA listings to securities master database
quantrocket.master: INFO Collecting CBOE IND listings from IB website (VIX only)
quantrocket.master: INFO Requesting details for 1 CBOE listings found on IB website
quantrocket.master: INFO Saved 1 CBOE listings to securities master database
quantrocket.master: INFO Collecting NYSE IND listings from IB website (TRIN-NYSE only)
quantrocket.master: INFO Requesting details for 1 NYSE listings found on IB website
quantrocket.master: INFO Saved 1 NYSE listings to securities master database
```

## Lookup ConIds

Look up the ConIds for the various instruments:

In [3]:
from quantrocket.master import download_master_file
download_master_file("indices.csv", exchanges=["CBOE", "NYSE", "ARCA"], symbols=["SPY", "VIX", "TRIN-NYSE"], sec_types=["IND","ETF"])

In [4]:
import pandas as pd
indices = pd.read_csv("indices.csv")

indices[["ConId", "Symbol", "LongName", "PrimaryExchange"]]

,ConId,Symbol,LongName,PrimaryExchange
0,756733,SPY,SPDR S&P 500 ETF TRUST,ARCA
1,13455763,VIX,CBOE Volatility Index,CBOE
2,26718743,TRIN-NYSE,NYSE TRIN (OR ARMS) INDEX,NYSE


## Collect historical data

Next, we create a database for collecting 1-day bars for the indexes:

In [5]:
from quantrocket.history import create_db
create_db("market-1d", 
          conids=[
              756733,
              13455763, 
              26718743,
          ], 
          bar_size="1 day")

{'status': 'successfully created quantrocket.history.market-1d.sqlite'}

Then collect the data:

In [6]:
from quantrocket.history import collect_history
collect_history("market-1d")

{'status': 'the historical data will be collected asynchronously'}

Monitor flightlog for completion:

```
quantrocket.history: INFO [market-1d] Collecting history from IB for 3 securities in market-1d
quantrocket.history: INFO [market-1d] Saved 13302 total records for 3 total securities to quantrocket.history.market-1d.sqlite
```

***

## *Next Up*

Part 3: [Moonshot Strategy Code](Part3-Moonshot-Strategy-Code.ipynb)